# Import Librairies

In [ ]:
!pip install pymongo
!pip install pandas
!pip install python-dateutil

In [1]:
import os
import shutil
import pandas as pd
import logging
from datetime import datetime
from pymongo import MongoClient

# CREDENTIALS

In [3]:


USER    = os.getenv("MONGO_INITDB_ROOT_USERNAME", "admin")
PASSWORD  = os.getenv("MONGO_INITDB_ROOT_PASSWORD", "admin")
HOST    = "mongo"
PORT    = 27017
AUTH_DB = "admin"

DB         = "AIRBNB"
COLLECTION = "Paris_listings"




In [2]:

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s  %(levelname)s  %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)


# Connection to Mongo

In [4]:
def connect_mongo():
   
    uri = f"mongodb://{USER}:{PASSWORD}@{HOST}:{PORT}/?authSource={AUTH_DB}"
    logging.info(f"Connection MongoDB : {uri}")
    client = MongoClient(uri)
    db = client[DB]
    coll = db[COLLECTION]
    return coll

# Data Cleaning

In [ ]:
def clean_csv(csv_path):

    csv_path ="C:/Users/udossougbete/Desktop/NOSQLEXAM/Data/listings_Paris.csv"
    logging.info(f"  → is loading : {csv_path}")
    try:
     df = pd.read_csv(csv_path, sep=",", engine="python")
    except TypeError:
     df = pd.read_csv(csv_path, sep=",", engine="python")

    df=df.fillna('')


    records = df.to_dict(orient="records")
    logging.info(f"    • Prêt à insérer {len(records)} document(s).")
    return records

 


    

In [ ]:
def etl_for_loading():
   
    
    coll = connect_mongo()
    logging.info(f" Collection in '{DB}': {coll.database.list_collection_names()}")
   

    fname = os.path.basename(csv_path)
    logging.info(f"--- file being processed: {fname} ---")
    try:
            records = clean_csv(csv_path)

            logging.info(f"  → Ingestion in MongoDB: {len(records)} documents.")
            result = coll.insert_many(records)
            logging.info(f"    ✔️ {len(result.inserted_ids)} Ingested docs from {fname}.")

            

    except Exception as e:
            logging.error(f"    ❌ Error during the processing of {fname}: {e}")
           

 
   

# ─────────────────────────────────────────────────────────────────────────────────────
# EXÉCUTION DU PIPELINE
logging.info("=== DÉMARRAGE DU PIPELINE ETL À " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + " ===")
etl_pipeline_for_all_new_csv()
logging.info("=== FIN DU PIPELINE ETL À " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + " ===")
